In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime ,timedelta
from pymongo import MongoClient
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import TimeSeriesSplit

In [2]:
dfa=pd.read_parquet("d://royal_omex.parquet")

In [3]:
le = LabelEncoder()
dfa['sensor'] = le.fit_transform(dfa['sensor'])

In [4]:
df1=dfa.groupby('sensor')

In [5]:
client = MongoClient("mongodb://localhost:27017/")
db = client["trial"]
col1=db['data1']
# col2=db['test_data']

In [13]:
# dit={}
for i, data in df1:
    df=data
    
    df.set_index('Clock',inplace=True)

    df=df.resample(rule="1H").sum()
    df=df[['Kwh']]
    df['Kwh']=df['Kwh']/1000
    df['sensor']=i

    today=datetime.today()
    today=today.replace(day=1,hour=0,minute=0,second=0, microsecond=0,month=11)#,year=2024,month=1)
    print(today)
    end_day = today.replace(day=1,hour=23) - timedelta(days=1)
    Start_day=end_day.replace(day=1,hour=0)
    df=df.loc[Start_day:end_day]
    df.reset_index(inplace=True)
    print("First day of last month:", Start_day)
    print("Last day of last month:", end_day)
    # dit[f"{i}"]=df
    col1.insert_one({f"{i}": df.to_dict(orient='records')})
# col1.insert_one(dit)
# dit


2023-11-01 00:00:00
First day of last month: 2023-10-01 00:00:00
Last day of last month: 2023-10-31 23:00:00
2023-11-01 00:00:00
First day of last month: 2023-10-01 00:00:00
Last day of last month: 2023-10-31 23:00:00


In [8]:
df.head(2)

,Clock,Kwh,sensor
0,2023-10-01 00:00:00,593.2,1
1,2023-10-01 01:00:00,542.3,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Clock   744 non-null    datetime64[ns]
 1   Kwh     744 non-null    float64       
 2   sensor  744 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 17.6 KB


In [11]:
sensor_key = "1"

result = col1.find_one({sensor_key: {"$exists": True}})

if result:
    data_array = result[sensor_key]
    df_retrieved = pd.DataFrame(data_array)
    
    print(df_retrieved)
else:
    print(f"No data found for sensor key: {sensor_key}")

                  Clock    Kwh  sensor
0   2023-10-01 00:00:00  593.2       1
1   2023-10-01 01:00:00  542.3       1
2   2023-10-01 02:00:00  518.0       1
3   2023-10-01 03:00:00  475.5       1
4   2023-10-01 04:00:00  458.8       1
..                  ...    ...     ...
739 2023-10-31 19:00:00  296.0       1
740 2023-10-31 20:00:00  297.0       1
741 2023-10-31 21:00:00  309.9       1
742 2023-10-31 22:00:00  294.5       1
743 2023-10-31 23:00:00  237.4       1

[744 rows x 3 columns]


In [15]:
df_retrieved

,Clock,Kwh,sensor
0,2023-10-01 00:00:00,593.2,1
1,2023-10-01 01:00:00,542.3,1
2,2023-10-01 02:00:00,518.0,1
3,2023-10-01 03:00:00,475.5,1
4,2023-10-01 04:00:00,458.8,1
...,...,...,...
739,2023-10-31 19:00:00,296.0,1
740,2023-10-31 20:00:00,297.0,1
741,2023-10-31 21:00:00,309.9,1
742,2023-10-31 22:00:00,294.5,1


In [14]:
df_retrieved.to_dict(orient='records')

[{'Clock': Timestamp('2023-10-01 00:00:00'), 'Kwh': 593.2, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 01:00:00'), 'Kwh': 542.3, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 02:00:00'), 'Kwh': 518.0, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 03:00:00'), 'Kwh': 475.5, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 04:00:00'), 'Kwh': 458.8, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 05:00:00'), 'Kwh': 451.1, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 06:00:00'), 'Kwh': 438.8, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 07:00:00'), 'Kwh': 411.0, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 08:00:00'), 'Kwh': 390.2, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 09:00:00'), 'Kwh': 385.9, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 10:00:00'), 'Kwh': 354.3, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 11:00:00'), 'Kwh': 342.0, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 12:00:00'), 'Kwh': 266.5, 'sensor': 1},
 {'Clock': Timestamp('2023-10-01 13:00:00'), 'Kwh':

In [19]:
df=df_retrieved[0:24]
df

,Clock,Kwh,sensor
0,2023-10-01 00:00:00,593.2,1
1,2023-10-01 01:00:00,542.3,1
2,2023-10-01 02:00:00,518.0,1
3,2023-10-01 03:00:00,475.5,1
4,2023-10-01 04:00:00,458.8,1
5,2023-10-01 05:00:00,451.1,1
6,2023-10-01 06:00:00,438.8,1
7,2023-10-01 07:00:00,411.0,1
8,2023-10-01 08:00:00,390.2,1
9,2023-10-01 09:00:00,385.9,1


In [20]:
l=[]
d={}
for value in df['Kwh']:
    d["{}".format(value)]=value
    l.append(d)
    

In [21]:
d

{'593.2': 593.2,
 '542.3': 542.3,
 '518.0': 518.0,
 '475.5': 475.5,
 '458.8': 458.8,
 '451.1': 451.1,
 '438.8': 438.8,
 '411.0': 411.0,
 '390.2': 390.2,
 '385.9': 385.9,
 '354.3': 354.3,
 '342.0': 342.0,
 '266.5': 266.5,
 '367.4': 367.4,
 '363.9': 363.9,
 '367.6': 367.6,
 '418.3': 418.3,
 '403.7': 403.7,
 '353.3': 353.3,
 '409.5': 409.5,
 '432.1': 432.1,
 '459.0': 459.0,
 '481.7': 481.7,
 '483.1': 483.1}

In [34]:
today=datetime.today()
today=today.replace(hour=0,minute=0,second=0, microsecond=0,)#month=1,year=2024)
print(today)
end_day = (today.replace(day=1,hour=23) - timedelta(days=1))
Start_day=end_day.replace(day=1,hour=0)

print("First day of month:", Start_day)
print("Last day of month:", end_day)

2023-12-11 00:00:00
First day of month: 2023-11-01 00:00:00
Last day of month: 2023-11-30 23:00:00


In [36]:
today = datetime.today()
today = today.replace(hour=0, minute=0, second=0, microsecond=0)
print(today)
end_day = (today.replace(day=1, hour=23) - timedelta(days=1))
start_day = end_day.replace(day=1, hour=0)

print("First day of month:", start_day)
print("Last day of month:", end_day)

2023-12-11 00:00:00
First day of month: 2023-11-01 00:00:00
Last day of month: 2023-11-30 23:00:00


In [48]:
today = datetime.today()
today = today.replace(hour=0, minute=0, second=0, microsecond=0)
first_day_of_month = today.replace(day=1)
last_day_of_month = (first_day_of_month.replace(month=first_day_of_month.month % 12 + 1, day=1) - timedelta(days=1))
last_day_of_month=last_day_of_month.replace(hour=23)
print("First day of the current month:", first_day_of_month)
print("Last day of the current month:", last_day_of_month)


First day of the current month: 2023-12-01 00:00:00
Last day of the current month: 2022-12-31 23:00:00
